# PlantDoc Image Classification Project

This notebook contains the main experiments and analysis for the PlantDoc dataset classification project.

## Table of Contents
1. [Setup and Data Loading](#setup)
2. [Data Preprocessing](#preprocessing)
3. [Traditional Machine Learning (HOG + SVM)](#traditional_ml)
4. [Deep Learning (ResNet Transfer Learning)](#deep_learning)
5. [Hyperparameter Sensitivity Analysis](#hyperparameter)
6. [Model Architecture Comparison](#architecture)
7. [Data Augmentation Analysis](#augmentation)
8. [Ablation Studies](#ablation)
9. [Failure Analysis](#failure)
10. [Model Interpretability](#interpretability)
11. [Final Results and Report](#results)

## 1. Setup and Data Loading <a name="setup"></a>

In [ ]:
# Import necessary libraries
import sys
import os
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report

# Import our custom modules
from download_data import download_plantdoc_dataset
from preprocess import load_plantdoc_data, create_train_val_test_split, create_data_generators, save_test_data
from traditional_ml import extract_hog_features, train_svm_classifier, evaluate_model
from deep_learning import create_resnet_model, compile_model, train_model, plot_training_history
from analysis import plot_confusion_matrix, analyze_confusion_matrix, plot_feature_maps, plot_hyperparameter_sensitivity

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

In [ ]:
# Download and load the dataset
print("Downloading PlantDoc dataset...")
data_path = download_plantdoc_dataset()

print("Loading dataset...")
images, labels, class_names = load_plantdoc_data(data_path)

print(f"Dataset loaded: {len(images)} images, {len(class_names)} classes")
print(f"Classes: {class_names}")

## 2. Data Preprocessing <a name="preprocessing"></a>

In [ ]:
# Split the data
X_train, X_val, X_test, y_train, y_val, y_test = create_train_val_test_split(images, labels)

# Save test data for final evaluation
save_test_data(X_test, y_test, class_names)

# Create data generators with augmentation
train_generator_aug, val_generator = create_data_generators(X_train, y_train, X_val, y_val, augmentation=True)

# Create data generators without augmentation for comparison
train_generator_no_aug, _ = create_data_generators(X_train, y_train, X_val, y_val, augmentation=False)

In [ ]:
# Visualize some sample images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i in range(10):
    ax = axes[i//5, i%5]
    ax.imshow(X_train[i])
    ax.set_title(f"Class: {class_names[y_train[i]]}")
    ax.axis('off')
plt.tight_layout()
plt.show()

## 3. Traditional Machine Learning (HOG + SVM) <a name="traditional_ml"></a>

In [ ]:
# Extract HOG features
print("Extracting HOG features from training data...")
X_train_hog = extract_hog_features(X_train)
X_val_hog = extract_hog_features(X_val)
X_test_hog = extract_hog_features(X_test)

print(f"HOG features shape: {X_train_hog.shape}")

In [ ]:
# Train SVM classifier
print("Training SVM classifier...")
svm_classifier, scaler = train_svm_classifier(X_train_hog, y_train, C=1.0, kernel='rbf')

# Evaluate on validation set
val_accuracy, val_report, val_predictions = evaluate_model(svm_classifier, scaler, X_val_hog, y_val, class_names)
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(val_report)

In [ ]:
# Evaluate on test set
test_accuracy, test_report, test_predictions = evaluate_model(svm_classifier, scaler, X_test_hog, y_test, class_names)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(test_report)

# Store results for comparison
traditional_ml_results = {
    'accuracy': test_accuracy,
    'predictions': test_predictions,
    'report': test_report
}

## 4. Deep Learning (ResNet Transfer Learning) <a name="deep_learning"></a>

In [ ]:
# Create ResNet model
num_classes = len(class_names)
resnet_model = create_resnet_model(num_classes)
resnet_model = compile_model(resnet_model)

print("ResNet model summary:")
resnet_model.summary()

In [ ]:
# Train the model
print("Training ResNet model...")
history = train_model(resnet_model, train_generator_aug, val_generator, epochs=50, model_name='resnet')

# Plot training history
plot_training_history(history, 'ResNet')

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = resnet_model.evaluate(val_generator)  # Using val_generator for consistency
print(f"ResNet Test Accuracy: {test_accuracy:.4f}")

# Get predictions for analysis
resnet_predictions = resnet_model.predict(val_generator)
resnet_pred_classes = np.argmax(resnet_predictions, axis=1)

# Store results
resnet_results = {
    'accuracy': test_accuracy,
    'predictions': resnet_pred_classes,
    'model': resnet_model
}

## 5. Hyperparameter Sensitivity Analysis <a name="hyperparameter"></a>

In [ ]:
# Learning rate sensitivity for ResNet
learning_rates = [1e-5, 1e-4, 1e-3, 1e-2]
lr_results = {}

for lr in learning_rates:
    print(f"Testing learning rate: {lr}")
    model = create_resnet_model(num_classes)
    model = compile_model(model, learning_rate=lr)
    
    # Quick training for sensitivity analysis
    history = model.fit(train_generator_aug, epochs=10, validation_data=val_generator, verbose=0)
    
    val_acc = max(history.history['val_accuracy'])
    val_loss = min(history.history['val_loss'])
    
    lr_results[lr] = {'accuracy': val_acc, 'loss': val_loss}

plot_hyperparameter_sensitivity(lr_results, 'Learning Rate', 'ResNet')

## 6. Model Architecture Comparison <a name="architecture"></a>

In [ ]:
# Compare ResNet50 vs VGG16
from deep_learning import create_vgg_model

# Train VGG16 model
vgg_model = create_vgg_model(num_classes)
vgg_model = compile_model(vgg_model)

print("Training VGG16 model...")
vgg_history = train_model(vgg_model, train_generator_aug, val_generator, epochs=30, model_name='vgg')

# Evaluate VGG16
vgg_loss, vgg_accuracy = vgg_model.evaluate(val_generator)
print(f"VGG16 Test Accuracy: {vgg_accuracy:.4f}")

# Compare models
model_comparison = {
    'SVM': {'accuracy': traditional_ml_results['accuracy']},
    'ResNet50': {'accuracy': resnet_results['accuracy']},
    'VGG16': {'accuracy': vgg_accuracy}
}

plot_model_comparison(model_comparison, 'accuracy')

## 7. Data Augmentation Analysis <a name="augmentation"></a>

In [ ]:
# Train ResNet without augmentation
resnet_no_aug = create_resnet_model(num_classes)
resnet_no_aug = compile_model(resnet_no_aug)

history_no_aug = resnet_no_aug.fit(train_generator_no_aug, epochs=30, validation_data=val_generator, verbose=0)
no_aug_accuracy = max(history_no_aug.history['val_accuracy'])

# Compare with augmented version
augmentation_results = {
    'no_augmentation': {'accuracy': no_aug_accuracy, 'loss': min(history_no_aug.history['val_loss'])},
    'with_augmentation': {'accuracy': resnet_results['accuracy'], 'loss': history.history['val_loss'][-1]}
}

plot_data_augmentation_comparison(augmentation_results['no_augmentation'], 
                                augmentation_results['with_augmentation'], 
                                'ResNet')

## 8. Ablation Studies <a name="ablation"></a>

In [ ]:
# Ablation study: Different optimizers
optimizers = ['adam', 'sgd', 'rmsprop']
optimizer_results = {}

for opt_name in optimizers:
    print(f"Testing optimizer: {opt_name}")
    model = create_resnet_model(num_classes)
    
    if opt_name == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    elif opt_name == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)
    elif opt_name == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4)
    
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(train_generator_aug, epochs=15, validation_data=val_generator, verbose=0)
    val_acc = max(history.history['val_accuracy'])
    
    optimizer_results[opt_name] = {'accuracy': val_acc}

# Plot ablation results
plt.figure(figsize=(8, 5))
opt_names = list(optimizer_results.keys())
opt_accs = [optimizer_results[opt]['accuracy'] for opt in opt_names]
plt.bar(opt_names, opt_accs)
plt.title('Ablation Study: Optimizer Comparison')
plt.ylabel('Validation Accuracy')
plt.savefig('reports/ablation_optimizers.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Failure Analysis <a name="failure"></a>

In [ ]:
# Confusion matrix for ResNet
plot_confusion_matrix(y_val, resnet_pred_classes, class_names, 'ResNet')

# Analyze most confused classes
confused_pairs = analyze_confusion_matrix(y_val, resnet_pred_classes, class_names)

## 10. Model Interpretability <a name="interpretability"></a>

In [ ]:
# Visualize feature maps from ResNet
# Preprocess a sample image
sample_img = X_val[0] / 255.0  # Normalize

# Visualize feature maps from different layers
layer_names = ['conv1_conv', 'conv2_block3_out', 'conv3_block4_out']

for layer_name in layer_names:
    try:
        plot_feature_maps(resnet_model, sample_img, layer_name, 'ResNet')
    except:
        print(f"Could not visualize layer: {layer_name}")

## 11. Final Results and Report <a name="results"></a>

In [ ]:
# Generate final test predictions for submission
print("Generating final test predictions...")

# Load test data
X_test_final = np.load('data/processed/X_test.npy')
y_test_final = np.load('data/processed/y_test.npy')

# Get predictions from best model (ResNet)
# Preprocess test images for ResNet
X_test_resnet = X_test_final / 255.0
test_predictions_resnet = resnet_model.predict(X_test_resnet)
test_pred_classes = np.argmax(test_predictions_resnet, axis=1)

# Save predictions
np.save('models/test_predictions.npy', test_pred_classes)

# Generate classification report
from analysis import generate_classification_report
final_report = generate_classification_report(y_test_final, test_pred_classes, class_names, 'ResNet_Final')

print("Final Results:")
print(f"Test Accuracy: {accuracy_score(y_test_final, test_pred_classes):.4f}")
print("\nClassification Report:")
print(final_report)

In [ ]:
# Summary of all experiments
print("\n=== EXPERIMENT SUMMARY ===")
print(f"Traditional ML (HOG+SVM) Accuracy: {traditional_ml_results['accuracy']:.4f}")
print(f"ResNet50 Accuracy: {resnet_results['accuracy']:.4f}")
print(f"VGG16 Accuracy: {vgg_accuracy:.4f}")
print(f"ResNet without augmentation: {no_aug_accuracy:.4f}")
print(f"ResNet with augmentation: {resnet_results['accuracy']:.4f}")

print("\nAll results and visualizations saved in the 'reports/' directory.")
print("Test predictions saved in 'models/test_predictions.npy'.")
print("\nNext steps:")
print("1. Write the experiment report based on these results.")
print("2. Create GitHub repository and upload code.")
print("3. Submit deliverables to guangyu.ryan@yahoo.com.")